# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Yz

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
import pickle

import nltk
nltk.download(["punkt", "wordnet", "stopwords"])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///emergency_messages.db')
df = pd.read_sql("SELECT * FROM df", engine)
X = df["message"]
Y = df.drop(["id", "message", "original", "genre"], axis=1) 

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-z0-9]", " ", text.lower().strip()) # normalize
    tokens = word_tokenize(text) # tokenize
    
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    clean_tokens = []
    for tok in tokens:
        if tok not in stopwords.words("english"):
            
            clean_tok = stemmer.stem(lemmatizer.lemmatize(lemmatizer.lemmatize(tok, pos="v")))
            clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

In [6]:
Y_pred = pd.DataFrame(Y_pred, columns=Y.columns)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
for i, var in enumerate(Y.columns):
    print(var)
    print(classification_report(Y_test.iloc[:,i], Y_pred.iloc[:,i]))

related_1
             precision    recall  f1-score   support

          0       0.63      0.51      0.56      1253
          1       0.85      0.91      0.88      3991

avg / total       0.80      0.81      0.80      5244

related_2
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5204
          1       0.46      0.28      0.34        40

avg / total       0.99      0.99      0.99      5244

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4318
          1       0.80      0.46      0.58       926

avg / total       0.88      0.88      0.87      5244

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5215
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
        'vect__max_df': (0.65, 0.75),
        #'tfidf__use_idf': (True, False),
        'clf__estimator__max_depth': [5, 6], 
        'clf__estimator__min_samples_leaf':[3, 4]  
    }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)

cv.fit(X_train, Y_train)
Y_pred_grid = cv.predict(X_test)

print(cv.best_params_)

{'clf__estimator__max_depth': 5, 'clf__estimator__min_samples_leaf': 3, 'vect__max_df': 0.75}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
Y_pred_grid = pd.DataFrame(Y_pred_grid, columns=Y.columns)

for i, var in enumerate(Y.columns):
    print(var)
    print(classification_report(Y_test.iloc[:,i], Y_pred_grid.iloc[:,i]))

related_1
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1253
          1       0.76      1.00      0.86      3991

avg / total       0.58      0.76      0.66      5244

related_2
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5204
          1       0.00      0.00      0.00        40

avg / total       0.98      0.99      0.99      5244

request
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      4318
          1       0.00      0.00      0.00       926

avg / total       0.68      0.82      0.74      5244

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5215
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.59      1.00      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [10]:
filename = 'emergency_message_model.sav'
pickle.dump(cv, open(filename, 'wb'))

NameError: name 'model' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.